In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227371
Azure region: eastus2
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-227371


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mimi-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {'--C': uniform(0.05, 0.5),
    '--max_iter': choice(5, 10, 50, 100)
    } ) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cluster_name,
                      environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10) ### YOUR CODE HERE ###

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

experiment= Experiment(ws, "hyper-mimi")
run = experiment.submit(config= hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6
Web View: https://ml.azure.com/runs/HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-227371/workspaces/quick-starts-ws-227371&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-08T05:10:54.654063][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space
[2023-03-08T05:10:55.2536551Z][SCHEDULER][INFO]Scheduling job, id='HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_0' 
[2023-03-08T05:10:55.3790982Z][SCHEDULER][INFO]Scheduling job, id='HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_1' 
[2023-03-08T05:10:55.4967336Z][SCHEDULER][INFO]Scheduling job, id='HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_2' 
[2023-03-08T05:10:55.6266662Z][SCHEDULER][INFO]Scheduling job, id='HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_3' 
[2023-03-08T05:10:55.8449070Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_

{'runId': 'HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6',
 'target': 'mimi-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-03-08T05:10:54.184585Z',
 'endTimeUtc': '2023-03-08T05:18:41.239697Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'f76ef773-3441-40b1-aefd-f307e1892b97',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.906409322651129',
  'best_child_run_id': 'HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_7',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

KeyError: 'log_files'

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run)
print('Accuracy:', best_run_metrics['Accuracy'])

Run(Experiment: hyper-mimi,
Id: HD_0f174e9d-c3f5-4e67-ab15-8509f29841d6_7,
Type: azureml.scriptrun,
Status: Completed)
Accuracy: 0.906409322651129


In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
    

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

train_df = x
train_df['y'] = y

In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###


experiment= Experiment(ws, "automl-mimi")
auto_run = experiment.submit(config= automl_config)
RunDetails(auto_run).show()
auto_run.wait_for_completion(show_output = True)

2023-03-08:05:36:57,334 WARNING  [local_experiment_driver.py:147] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl-mimi,AutoML_f13eb99d-896e-446b-b0a9-6f10449b9c4f,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/03/08 05:44:51 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-03-08:06:13:47,76 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-03-08:06:13:49,821 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-03-08:06:13:49,826 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-03-08:06:13:52,680 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-03-08:0

NameError: name 'RunDetails' is not defined

In [14]:
from azureml.widgets import RunDetails
RunDetails(auto_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [18]:
# Get your best run and save the model from that run.

best_auto_run = auto_run.get_best_child()
print(best_auto_run)

Run(Experiment: automl-mimi,
Id: AutoML_f13eb99d-896e-446b-b0a9-6f10449b9c4f_21,
Type: None,
Status: Completed)
